INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-ECB-en-fr"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd
os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-ECB-en-fr20210305-1002


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/ecb-en-fr.txt.zip
!mkdir data_bin
!unzip 'ecb-en-fr.txt.zip' -d 'data_bin'

--2021-03-05 10:02:01--  https://github.com/hoangtrungchinh/clc_data/raw/master/ecb-en-fr.txt.zip
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/ecb-en-fr.txt.zip [following]
--2021-03-05 10:02:01--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/ecb-en-fr.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18865629 (18M) [application/zip]
Saving to: ‘ecb-en-fr.txt.zip’

ecb-en-fr.txt.zip   100%[===================>]  17,99M  7,14MB/s    in 2,5s    

2021-03-05 10:02:06 (7,14 MB/s) - ‘ecb-en-fr.txt.zip’ saved [18865629/18865629]

Archive:

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
fr_files = sorted(glob.glob("data_bin/*.fr"))
print(en_files)
print(fr_files)

lst_en = []
lst_fr = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line.strip())

for file in fr_files:
  with open(file) as file_in:
    for line in file_in:
      lst_fr.append(line.strip())

print(len(lst_en))
print(len(lst_fr))

['data_bin/ECB.en-fr.en']
['data_bin/ECB.en-fr.fr']
195960
195960


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, fr_train, fr_test_valid =  train_test_split(lst_en, lst_fr, test_size=0.2, random_state=123)
en_valid, en_test, fr_valid, fr_test = train_test_split(en_test_valid, fr_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(fr_train))
print(len(en_valid))
print(len(fr_valid))
print(len(en_test))
print(len(fr_test))
print(en_train[20])
print(fr_train[20])
print(en_valid[-1])
print(fr_valid[-1])
print(en_test[-1])
print(fr_test[-1])

156768
156768
19596
19596
19596
19596
ECB / 2006/5
BCE / 2006/5
When the service-providing NCB receives a positive or negative acknowledgement from the receiving NCB / ECB it shall forward such an acknowledgement to the connected NCB . Transfer of responsibility for the execution of a payment order For cross-border payments initiated by a participant in the connected NCB 's RTGS system , responsibility for the execution of a payment order shall pass from the connected NCB to the service-providing NCB at the moment the connected NCB 's account with the service-providing NCB is debited , and thereafter it shall pass to the receiving NCB / ECB in accordance
Transfert de responsabilité pour l' exécution d' un ordre de paiement Pour les paiements transfrontaliers émis par un participant au système RBTR de la BCN connectée , la responsabilité de l' exécution d' un ordre de paiement est transférée de la BCN connectée à la BCN prestataire de services , au moment où le compte de la BCN connecté

In [6]:
file_variable = [en_train,fr_train,en_valid,fr_valid,en_test,fr_test]
file_names = ["en_train","fr_train","en_valid","fr_valid","en_test","fr_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
      f.write("\n")
    
!ls -al

total 87192
drwxrwxr-x 3 chinh chinh     4096 Thg 3   5 10:02 .
drwxrwxr-x 6 chinh chinh     4096 Thg 3   5 10:02 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3   5 10:02 data_bin
-rw-rw-r-- 1 chinh chinh 18865629 Thg 3   5 10:02 ecb-en-fr.txt.zip
-rw-rw-r-- 1 chinh chinh  3262752 Thg 3   5 10:02 en_test
-rw-rw-r-- 1 chinh chinh 25944504 Thg 3   5 10:02 en_train
-rw-rw-r-- 1 chinh chinh  3208009 Thg 3   5 10:02 en_valid
-rw-rw-r-- 1 chinh chinh  3798492 Thg 3   5 10:02 fr_test
-rw-rw-r-- 1 chinh chinh 30424593 Thg 3   5 10:02 fr_train
-rw-rw-r-- 1 chinh chinh  3754500 Thg 3   5 10:02 fr_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [9]:
start = time.time()
srcfile = "en_train"
tarfile = "fr_train"
lst_srcfile = en_train
lst_tarfile = fr_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  2519.969575405121


Test Cosin similarity

In [10]:
import faiss

start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  126.42454671859741
156768
156768
[[     0  80931  88493  17232 117646  55733]
 [     1  74231  87525  70790  57947 143081]
 [     2  98504 121309 121276  11306 127947]
 ...
 [156765  45924  21346  14694  88602 110434]
 [156766 132448 105187 153756  76885  34789]
 [ 99540  25656 156767 112645 153288  59683]]
[[0.0000000e+00 1.2904823e-01 1.3689876e-01 3.2998800e-01 3.4019959e-01
  3.8171685e-01]
 [0.0000000e+00 3.1483066e-01 3.1934476e-01 4.4118845e-01 4.4317710e-01
  4.5472217e-01]
 [2.3841858e-07 5.4844296e-01 5.6592822e-01 5.8806765e-01 6.1782336e-01
  6.2109041e-01]
 ...
 [0.0000000e+00 6.3668823e-01 6.9641459e-01 7.1334958e-01 7.2244561e-01
  7.2244561e-01]
 [2.3841858e-07 2.3160541e-01 2.5245798e-01 2.6616895e-01 2.9075861e-01
  2.9164243e-01]
 [0.0000000e+00 0.0000000e+00 1.1920929e-07 4.2252779e-02 5.3200126e-01
  5.9428740e-01]]


In [11]:
j = k-1
# TEST
for i in range(10):
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    dist_squared = np.sum(np.square(a - b))
    print("index:",index[i][j], "distance:",distance[i][j], "tested:",dist_squared)

index: 55733 distance: 0.38171685 tested: 0.38171703
index: 143081 distance: 0.45472217 tested: 0.45472252
index: 127947 distance: 0.6210904 tested: 0.62109053
index: 26752 distance: 0.4888314 tested: 0.4888316
index: 11732 distance: 0.39531517 tested: 0.39531568
index: 153074 distance: 0.55164826 tested: 0.5516484
index: 145125 distance: 0.55505645 tested: 0.5550566
index: 31983 distance: 0.6475519 tested: 0.64755183
index: 134388 distance: 0.50587094 tested: 0.5058708
index: 135842 distance: 0.53762674 tested: 0.5376268


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.9, 0.8, 0.7, 0.6]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i] + saperate + lst_tarfile[best_simi_index]

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i]
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)

        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)
    


threshold = 0.9 EM sentences = 377 / 156768 0.002404827515819555 %
threshold = 0.8 EM sentences = 1345 / 156768 0.008579557052459686 %
threshold = 0.7 EM sentences = 4662 / 156768 0.029738211879975506 %
threshold = 0.6 EM sentences = 13824 / 156768 0.08818126148193509 %
=== Ending, Total time (second):  14.66804027557373
